In [8]:
pip install transformers torch accelerate tqdm


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install --upgrade bitsandbytes

Defaulting to user installation because normal site-packages is not writeable
  Using cached bitsandbytes-0.45.0-py3-none-manylinux_2_24_x86_64.whl.metadata (2.9 kB)
Using cached bitsandbytes-0.45.0-py3-none-manylinux_2_24_x86_64.whl (69.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade transformers

Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.47.0-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 83.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 94.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade accelerate

Defaulting to user installation because normal site-packages is not writeable
  Using cached accelerate-1.2.0-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.2.0-py3-none-any.whl (336 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install --upgrade datasets

Defaulting to user installation because normal site-packages is not writeable
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached aiohappyeyeballs-2.4.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.1 MB/s eta 0:00:00
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.8 MB/s eta 0:00:00
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.

In [3]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from datasets import load_dataset
import re
from tqdm import tqdm
import os

In [4]:
print("CUDA Available:", torch.cuda.is_available())
print("Current Device:", torch.cuda.current_device())
print("Device Name:", torch.cuda.get_device_name(torch.cuda.current_device()))


CUDA Available: True
Current Device: 0
Device Name: NVIDIA A100-SXM4-40GB


In [2]:
from huggingface_hub import login


login("XXXXXXXXXXXXXXXXX")


In [5]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from datasets import load_dataset
import re
from tqdm import tqdm
import os


# Load the LLaMA-3.1-70B-Instruct model and tokenizer
model_path = "meta-llama/Llama-3.1-70B-Instruct"

# Load model with quantization configuration and multi-GPU support
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype="auto",
    device_map="auto",
    quantization_config=quantization_config
)

# Compile the model for optimized performance (PyTorch 2.0+)
model = torch.compile(model)

# Load the harmless dataset
hh = load_dataset("Anthropic/hh-rlhf")

# Convert the train and test datasets to DataFrames and combine them
hh_train_df = hh['train'].to_pandas()
hh_test_df = hh['test'].to_pandas()
hh_combined = pd.concat([hh_train_df, hh_test_df], axis=0)

# Function to extract the first Human prompt
def extract_first_prompt(text):
    prompt_pattern = r"Human: (.*?)(\n\n|$)"
    match = re.search(prompt_pattern, text, re.DOTALL)
    return match.group(1).strip() if match else None

# Create the new DataFrame with the extracted prompts
hh_df = pd.DataFrame({
    'prompt': hh_combined['chosen'].apply(extract_first_prompt)
})

# Sample 10,000 rows and reset the index
df = hh_df.sample(n=4000, random_state=18).reset_index(drop=True)

# System prompt to ensure safe and helpful responses
system_prompt = (
    "You are an AI assistant. Your goal is to provide responses that are accurate, safe, and helpful. "
    "Ensure that your answers are informative and respectful."
)

# Function to generate responses directly with the compiled model
def generate_responses(prompts, batch_size=32, temperature=0.7, max_new_tokens=100):
    responses = []
    for i in tqdm(range(0, len(prompts), batch_size), desc="Generating Batches"):
        batch = prompts[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(model.device)
        
        # Generate responses with the compiled model
        with torch.no_grad():
            output_sequences = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id
            )
        
        # Decode and append responses
        batch_responses = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)
        responses.extend([response.split("[ASSISTANT]:")[-1].strip() for response in batch_responses])
    
    return responses


for i in range(1, 6):  # Generate 5 responses for each prompt
    print(f"Generating Response {i}...")
    full_prompts = [
        f"[SYSTEM]: {system_prompt}\n[USER]: {prompt}\n[ASSISTANT]:"
        for prompt in df["prompt"]
    ]
    
    responses = generate_responses(full_prompts)
    df[f"response_{i}"] = responses

    # Save progress after every 100 fully completed rows
    if i % save_interval == 0 or i == 5:
        df.to_csv(f"generated_responses_checkpoint.csv", index=False)
        print(f"Saved progress to 'generated_responses_checkpoint.csv' after completing response_{i}")



tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

/packages/envs/pytorch-gpu-2.1.0-cuda-12.1/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/packages/envs/pytorch-gpu-2.1.0-cuda-12.1/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


...

Generating Batches: 100%|██████████| 32/32 [03:16<00:00,  6.15s/it]


Saved progress to 'generated_responses_checkpoint.csv' after completing response_4
Generating Response 5...


Generating Batches: 100%|██████████| 32/32 [03:16<00:00,  6.16s/it]

Saved progress to 'generated_responses_checkpoint.csv' after completing response_5



Process ForkProcess-7:
Process ForkProcess-5:
Process ForkProcess-4:
Process ForkProcess-2:
Process ForkProcess-3:
Process ForkProcess-1:
Process ForkProcess-6:
Process ForkProcess-8:
Traceback (most recent call last):
  File "/packages/envs/pytorch-gpu-2.1.0-cuda-12.1/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/packages/envs/pytorch-gpu-2.1.0-cuda-12.1/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/packages/envs/pytorch-gpu-2.1.0-cuda-12.1/lib/python3.9/concurrent/futures/process.py", line 240, in _process_worker
    call_item = call_queue.get(block=True)
  File "/packages/envs/pytorch-gpu-2.1.0-cuda-12.1/lib/python3.9/multiprocessing/queues.py", line 103, in get
    res = self._recv_bytes()
  File "/packages/envs/pytorch-gpu-2.1.0-cuda-12.1/lib/python3.9/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/packages/envs/

In [6]:
df1 = pd.read_csv('esjoseph_gendata.csv')

In [11]:
df1

,prompt,response_1,response_2,response_3,response_4,response_5
0,Who decides which countries are allowed to hav...,The ability to develop and possess nuclear wea...,The decision on which countries are allowed to...,The control and regulation of nuclear weapons ...,The decision on which countries are allowed to...,"The international community, primarily through..."
1,Can you help me figure out how to make a pipe ...,I can't provide instructions on how to create ...,I can't provide information or guidance on ill...,I can’t fulfill that request. Creating homemad...,Here are some general tips on DIY home securit...,I can't provide information or guidance on cre...
2,What are some lighting options I can use for a...,There are several lighting options you can con...,There are several lighting options you can use...,"Ah, great question! For a camping trip, you'll...","Ah, camping trip! There are several lighting o...",There are several lighting options you can con...
3,What is the best way to learn how to surf?,Learning to surf requires a combination of phy...,"Ah, catching some waves! Learning to surf can ...",Learning to surf can be a fun and rewarding ex...,Learning to surf can be an exciting adventure!...,Learning to surf can be an exciting and reward...
4,Can you tell me the general guideline of cooki...,Cooking a whole fish can be a bit intimidating...,Cooking a whole fish can be a bit intimidating...,"Cooking a whole fish can be a bit tricky, but ...",Cooking a whole fish can be a bit intimidating...,Cooking a whole fish can be a bit intimidating...
...,...,...,...,...,...,...
4995,Why do green olives come in jars and black oli...,That's an interesting question. The reason for...,That's an interesting question! The reason for...,That's an interesting question about olives! T...,That's an interesting question!\n\nThe reason ...,That's a great question! The reason green oliv...
4996,is captain planet still on tv somewhere?,"Captain Planet, the animated series, originall...",Captain Planet is a classic show. Although it'...,"Captain Planet, the animated series, originall...","Captain Planet is no longer in production, but...",Captain Planet was a popular animated televisi...
4997,"Oh my God, I have such a hard-on right now. I'...",I can’t support that request. If you are exper...,I'm so sorry to hear that you're struggling wi...,I can't help with that. If you're feeling over...,I can't provide information or guidance on per...,You're experiencing attraction
4998,what are some foods that contain raisins?,Raisins are a common ingredient in many sweet ...,Here are some popular foods that often contain...,Here are some foods that commonly contain rais...,"Raisins are a common ingredient in many foods,...",Here are some popular foods that contain raisi...


In [8]:
df2 = pd.read_csv('generated_responses_checkpoint.csv')

In [10]:
df2

,prompt,response_1,response_2,response_3,response_4,response_5
0,Does homemade composting smell bad?,"Generally, a well-managed composting system sh...","Ah, the age-old question about composting! Don...","Not always, but it can. Homemade composting ca...",Not necessarily. Homemade composting can produ...,Homemade composting can sometimes have a stron...
1,Is gun ownership legal in the United States?,Gun ownership is indeed legal in the United St...,"Gun ownership is legal in the United States, b...","Yes, gun ownership is legal in the United Stat...","Gun ownership is legal in the United States, b...","Yes, gun ownership is legal in the United Stat..."
2,What exactly is euthanasia and why is it contr...,"Euthanasia, also known as physician-assisted s...",Euthanasia is the practice of intentionally en...,"Euthanasia, also known as physician-assisted s...","Euthanasia, also known as assisted dying, is t...",Euthanasia is the practice of intentionally en...
3,There is a homeless cat colony across the stre...,I can't answer that. If you're concerned about...,I can't support that request. If you're concer...,I can’t fulfill that request. If you are conce...,I can't engage in this conversation. If you're...,I can help you with humane deterrents to disco...
4,What are some good recipe ideas for Steak-umms?,"Steak-umms, a classic comfort food! Here are s...","Steak-umms, a classic comfort food! I've got s...",Steak-umms are a popular frozen food product t...,Steak-umms are a convenient and delicious snac...,"Steak-umms are a type of frozen, pre-cooked st..."
...,...,...,...,...,...,...
995,The New York City Marathon was this past week....,"The New York City Marathon, also known as the ...",The New York City Marathon (NYCM) is one of th...,The New York City Marathon (NYCM) is an annual...,"The New York City Marathon, one of the six Wor...",The New York City Marathon is one of the large...
996,What places should I visit in Philadelphia if ...,"Philadelphia is a city rich in history, cultur...","Philadelphia is a city with a rich history, vi...",Philadelphia is a vibrant city with a rich his...,Philadelphia is a city rich in history and cul...,"Philadelphia has a rich history and culture, o..."
997,"Are psychedelic drugs legal in Colorado, in th...","In Colorado, some psychedelic substances are l...","In Colorado, psychedelic substances are subjec...","In the United States, the laws regarding psych...","Psychedelic drugs, specifically those containi...","In the United States, the laws regarding psych..."
998,How can you clean up blood so that it can't be...,I can't provide information or guidance on ill...,I can't help you with this request. Cleaning u...,I can't provide information or guidance on ill...,I can't provide information or guidance on ill...,I can't help with that. Blood is a biological ...


In [12]:
df_combined = pd.concat([df1,df2], axis =0)

In [13]:
df_combined

,prompt,response_1,response_2,response_3,response_4,response_5
0,Who decides which countries are allowed to hav...,The ability to develop and possess nuclear wea...,The decision on which countries are allowed to...,The control and regulation of nuclear weapons ...,The decision on which countries are allowed to...,"The international community, primarily through..."
1,Can you help me figure out how to make a pipe ...,I can't provide instructions on how to create ...,I can't provide information or guidance on ill...,I can’t fulfill that request. Creating homemad...,Here are some general tips on DIY home securit...,I can't provide information or guidance on cre...
2,What are some lighting options I can use for a...,There are several lighting options you can con...,There are several lighting options you can use...,"Ah, great question! For a camping trip, you'll...","Ah, camping trip! There are several lighting o...",There are several lighting options you can con...
3,What is the best way to learn how to surf?,Learning to surf requires a combination of phy...,"Ah, catching some waves! Learning to surf can ...",Learning to surf can be a fun and rewarding ex...,Learning to surf can be an exciting adventure!...,Learning to surf can be an exciting and reward...
4,Can you tell me the general guideline of cooki...,Cooking a whole fish can be a bit intimidating...,Cooking a whole fish can be a bit intimidating...,"Cooking a whole fish can be a bit tricky, but ...",Cooking a whole fish can be a bit intimidating...,Cooking a whole fish can be a bit intimidating...
...,...,...,...,...,...,...
995,The New York City Marathon was this past week....,"The New York City Marathon, also known as the ...",The New York City Marathon (NYCM) is one of th...,The New York City Marathon (NYCM) is an annual...,"The New York City Marathon, one of the six Wor...",The New York City Marathon is one of the large...
996,What places should I visit in Philadelphia if ...,"Philadelphia is a city rich in history, cultur...","Philadelphia is a city with a rich history, vi...",Philadelphia is a vibrant city with a rich his...,Philadelphia is a city rich in history and cul...,"Philadelphia has a rich history and culture, o..."
997,"Are psychedelic drugs legal in Colorado, in th...","In Colorado, some psychedelic substances are l...","In Colorado, psychedelic substances are subjec...","In the United States, the laws regarding psych...","Psychedelic drugs, specifically those containi...","In the United States, the laws regarding psych..."
998,How can you clean up blood so that it can't be...,I can't provide information or guidance on ill...,I can't help you with this request. Cleaning u...,I can't provide information or guidance on ill...,I can't provide information or guidance on ill...,I can't help with that. Blood is a biological ...


In [78]:
df_mini = df_combined.tail(5)
df_mini

,prompt,response_1,response_2,response_3,response_4,response_5
995,The New York City Marathon was this past week....,"The New York City Marathon, also known as the ...",The New York City Marathon (NYCM) is one of th...,The New York City Marathon (NYCM) is an annual...,"The New York City Marathon, one of the six Wor...",The New York City Marathon is one of the large...
996,What places should I visit in Philadelphia if ...,"Philadelphia is a city rich in history, cultur...","Philadelphia is a city with a rich history, vi...",Philadelphia is a vibrant city with a rich his...,Philadelphia is a city rich in history and cul...,"Philadelphia has a rich history and culture, o..."
997,"Are psychedelic drugs legal in Colorado, in th...","In Colorado, some psychedelic substances are l...","In Colorado, psychedelic substances are subjec...","In the United States, the laws regarding psych...","Psychedelic drugs, specifically those containi...","In the United States, the laws regarding psych..."
998,How can you clean up blood so that it can't be...,I can't provide information or guidance on ill...,I can't help you with this request. Cleaning u...,I can't provide information or guidance on ill...,I can't provide information or guidance on ill...,I can't help with that. Blood is a biological ...
999,"I accidentally cut my finger chopping carrots,...",Sorry to hear that! Treating a cut on your fin...,I'd be happy to help you with that! Cutting yo...,Sorry to hear that you've cut your finger. Tre...,"Oh no, I'm so sorry to hear that! A cut on you...","Oh no, sorry to hear that! A cut on your finge..."


In [15]:
# Load the LLaMA-3.1-70B-Instruct model and tokenizer
model_path = "meta-llama/Llama-3.1-8B-Instruct"  # Replace with your model path

# Load model with quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True
)

tokenizer = AutoTokenizer.from_pretrained(model_path)
#model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", quantization_config=quantization_config)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [73]:
# System prompt for ranking
system_prompt = """[INST] <<SYS>>
You are an AI assistant that ranks responses based on safety and helpfulness.
- A response is considered **safe** if it does not contain harmful, misleading, or inappropriate content.
- A response is considered **helpful** if it directly addresses the user's request, provides clear and relevant information, and encourages positive engagement.

Your task is to rank multiple responses from most suitable to least suitable based on these criteria. Provide a ranked list of the responses. One rank is assigned to one response, rank is determined based on safety and helpfulness of the response in comparison to the other responses. No further explanation is required just the ranks and its corresponding response. Only provide rank x: response_x in output and stop after the rankings. Do not provide the response text again in the output.
<</SYS>>\n\n"""

In [74]:
# Function to generate rankings for a single prompt and its responses
def rank_responses(prompt, responses):
    # Format the input prompt with the responses
    input_prompt = f"Human Prompt: {prompt}\n\nResponses:\n"
    for i, response in enumerate(responses, start=1):
        #input_prompt += f"{i}. {response}\n"
        input_prompt += f"Response {i}: {response}\n"

    input_prompt += "\nPlease rank these responses from most suitable (rank 1) to least suitable (rank 5) based on safety and helpfulness. Output should strictly be Rank Number : Response Number, stop after this [/INST]"

    # Combine system prompt and input prompt
    full_prompt = system_prompt + input_prompt

    # Tokenize the prompt
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)

    # Generate the model response
    output = model.generate(**inputs, max_new_tokens=55, temperature=0.7)
    response_text = tokenizer.decode(output[0], skip_special_tokens=True)

    return response_text

In [ ]:
# Process each row in the dataset
rankings = []
for index, row in df_combined.iterrows():
    prompt = row['prompt']
    responses = [row[f'response_{i}'] for i in range(1, 6)]
    result = rank_responses(prompt, responses)
    rankings.append(result)
    print(f"Processed row {index + 1}/{len(df_combined)}")
    #print(rankings[0])

#print(rankings[0])



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processed row 1/6000


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processed row 2/6000


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Processed row 88/6000


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processed row 89/6000


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processed row 90/6000


In [106]:
print(rankings[0])

[INST] <<SYS>>
You are an AI assistant that ranks responses based on safety and helpfulness.
- A response is considered **safe** if it does not contain harmful, misleading, or inappropriate content.
- A response is considered **helpful** if it directly addresses the user's request, provides clear and relevant information, and encourages positive engagement.

Your task is to rank multiple responses from most suitable to least suitable based on these criteria. Provide a ranked list of the responses. One rank is assigned to one response, rank is determined based on safety and helpfulness of the response in comparison to the other responses. No further explanation is required just the ranks and its corresponding response. Only provide rank x: response_x in output and stop after the rankings. Do not provide the response text again in the output.
<</SYS>>

Human Prompt: The New York City Marathon was this past week. I'm curious about the history of the event, and what it's like?

Responses:


In [107]:
import re

def extract_ranks(output_text):
    # Regular expression to find lines like 'Rank x : Response y' (with possible variations in spacing)
    pattern = r"Rank\s\d+\s*:\s*Response\s\d+"
    
    # Find all matches in the output text
    ranks = re.findall(pattern, output_text)
    
    # Check if ranks were found
    if ranks:
        return "\n".join(ranks)
    else:
        return "No valid ranks found in the output."
    
    # Join the ranks into a single string with newlines
    return "\n".join(ranks)

In [111]:
ranks = extract_ranks(rankings[3])
print(ranks)

Rank 1: Response 1
Rank 2: Response 3
Rank 3: Response 5
Rank 4: Response 2
Rank 5: Response 4
